In [50]:
import numpy as np
import networkx as nx
from shapely import Polygon, MultiPoint

In [292]:
with open("../data/12.txt") as f:
    data = [line.rstrip() for line in f.readlines()]

nr = len(data)
nc = len(data[0])
data = np.array([dd for d in data for dd in d]).reshape((nr,nc))
assert nr == nc

In [ ]:
def get_idx(r,c,nc=nc):
    return (r*nc)+c

def invert_idx(idx,nc,nr):
    return (idx//nr,idx%nc)

directions = [
    (-1,0), 
    (1,0),
    (0,1),
    (0,-1)
]

def take_step(i,step,data=data):
    nr,nc = data.shape
    move = (i[0]+step[0],i[1]+step[1])
    if move[0]>=0 and move[1]>=0 and move[0]<nc and move[1]<nr:
        return move

def convert_to_bbox(idx):
# idx = 0
    r,c = invert_idx(idx,nr,nc)
    box = [(r,c),(r+1,c),(r,c+1),(r+1,c+1)]
    bbox = [r,c+1,r+1,c]
    return bbox

In [294]:


g = nx.Graph()

# add all the nodes
i=0
for r in range(nr):
    for c in range(nc):
        assert i == get_idx(r,c,nc)
        g.add_node(i,plant=data[(r,c)])
        i+=1

# add edges
for r in range(nr):
    for c in range(nc):
        steps = [take_step((r,c),d,data) for d in directions ]
        for step in steps:
            if step:
                if data[step] == data[(r,c)]:
                    g.add_edge(get_idx(r,c),get_idx(*step))


In [295]:
prices = []
for region in list(nx.connected_components(g)):
    perimeter = sum([4 - g.degree[n] for n in region])
    area = len(region)
    prices.append(perimeter * area)
sum(prices)

1464678

In [296]:
# part two

In [ ]:
discounted_prices = 0
for region in nx.connected_components(g):
    boxes = [shapely.geometry.box(*convert_to_bbox(r)) for r in region]
    poly = boxes[0]
    for b in boxes[1:]:
        poly = poly.union(b)
    sides = 0
    for pr in shapely.get_rings(poly):
        coords = [c for c in shapely.simplify(pr,0).coords]
        sides += len(set(coords))
    discounted_prices += (len(region) * sides)
print(discounted_prices)  
